# Gaia Association
** **
## Example Notebook and User Guide
** **
** **
![alternatvie text](https://raw.githubusercontent.com/samrosean/images/main/logo_with_border_transparent.png)

## Step 1: Import Neccesary Libraries: 

gaiaAssociation is built to be dependant on a few python packages including pandas, pyranges, numpy, scipy, and seaborn, as well as built-in python packages like sys and os. These packages will need to be loaded into your environment if you wish to use gaiaAssociation. The cell below will import all neccessary packages, but it may present errors if these packages are not already installed in your copy of python. To install a package you can run a code cell with:

*"pip install \[package-name]"*

or if working through an anaconda environment

*"conda install \[package-name]"*

If this fails, each package will have its own troubleshooting documentation available to help install it into your environment.

In [6]:
import pandas as pd
import pyranges as pr
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import operator
import scipy
from scipy.cluster.hierarchy import dendrogram, leaves_list
import scipy.spatial.distance as ssd
import os, glob
import sys
import seaborn as sns
import statistics
import math
import argparse
from scipy.optimize import root
from scipy.stats import norm

## Step 2: Import gaiaAssociation: 

Just as you have done in the previous step, gaiaAssociation will have to be installed into your local environment, this can be done easily by using pip install to retreive it from the PyPi repository. After it has been installed, it can be imported, at which point it is now usable within this notebook!

In [ ]:
pip install gaiaAssociation

In [2]:
import gaiaAssociation.gaiaAssociation

## Step 3: Using gaiaAssociation: 

### Required variables:

gaiaAssociation has 4 required variables which must be given. 

1. **OCR Folder**: the folder location of the OCR bed files stored in .txt format. The first three columns should be labeled “Chromosome”, “Start”, and “End” in that order. 

                ex: "/User/OCR_Files"


2. **Loci Folder**: the folder location of the loci files stored in .tsv and/or .csv format.These files can be formatted differently depending on the type of loci being compared. If these loci are a single base pair long then only two columns are required: “CHR_POS” and “CHR_ID” which refer to its genome location and its chromosome name respectively. If these are variably sized loci you can include “Start” and “End” instead of “CHR_POS”. If you would like to give specific labels to each loci set, since by default we will name the loci set by their filename, you can include a column titled "DISEASE/TRAIT", which allows for multiple different loci sets to be analyzed within one file.

                ex: "/User/loci_Files"

3. **Chromosome Size**: The location of a chromosome size file stored in a .csv format. This should only have two columns “Chromosome” and “size in bp”. This file can be used to subset what chromosomes you are interested in looking at enrichment within. If you only include chr1 and chr2, then enrichment will only be done relative to these two chromosomes. It is important to give it a path including the file, not just the path. (e.g. user/chrom/chrsize.csv not just user/chrom)

                ex: "/User/Chrom/chrsize.csv"


4. **Output Folder**: The folder location you want the results to be output into. If this folder does not already exist gaia will attempt to make it. If it does not have the permissions to do so, it will exit and the user will have to run it with folder creating permissions, or they will have to make the folder themselves before running again.

                ex: "/User/Output"
                

A default run using just these required arguments on the attached data is given below:

In [4]:
gaiaAssociation.gaiaAssociation.gaiaAssociation("/User/OCR_Fules", "/User/loci_Files", "/User/Chrom/chrsize.csv", "/User/Output")

SystemExit: Loci folder cannot be found

/Users/samuelrosean/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### Optional Arguments

gaiaAssociation also has five optional arguments which can be given to generate more specific results.

1. **Peak Uniqueness**: a cutoff value for OCR uniqueness (e.g. if given 12, then any atac peak found in more than 12 atac sets will be removed from all of them) - by default uniqueness is not considered).

            ex: 12
2. **Loci Cutoff**: a loci cutoff value, when given to gaiaAssociation, the run will only consider loci groups (phenotypes or cohorts) with more loci than this cutoff value - by default this cutoff value is 0.

            ex: 200

3. **Specific Loci**: a tsv file with the identifiers of specific loci groups you would like to use. This can be very helpful if using a large loci set with with many phenotypes, and you want to sort by more than just loci count. The tsv should have column names which also exist in the loci files, so that gaia can subset the loci based on these column values (e.g. if you have a column in your loci set titled "runs" and you would like to only use run 1, then a tsv file with one column "runs" and one row value "1" will accomplish this). If the tsv file includes multiple columns, say "STUDY", and "DISEASE/TRAIT", then the loci set will be subset specifically by both columns at once. So only those that match every value within a row will be considered as a group (e.g. if you subset by "runs" and by "patient", then if only those loci that match both of these values will be considered, so perhaps only Patient 1 from Run 1).

            ex: "/User/LociSelection/LociSelection.tsv"

4. **Masking Region**: a bed file in  a .txt format containing a set of regions which you would like to subset every OCR region by. For example, a set of regions around the TSSs of a list of particular genes. This will reduce the OCR regions to just those that overlap with this given set of regions. This can be used to detect cell-specific + site-specific enrichment differences.

            ex: "/User/Masking/MaskedRegion.txt"

5. **Window Size**: an integer given to represent the size of windows the user would like to divide the chromosome into. This method is based on the sinib tool (https://github.com/boxiangliu/sinib), which requires the chromosome be divided into a series of equal length windows to be able to model them as a series of binomials. The default value is 100,000 bp, but this value can be changed to increase specificity or decrease sepcificity. The function of the window size is to only consider the local environment when determing loci enrichment, so consideration should be made to what the user considers local in their particualr context.

            ex: 1000000000

### Contact: 

For problems with gaiaAssociation, you can raise an issue through the github page, or contact me at:
    
    samuel.rosean@einsteinmed.edu

** **
** **

## Extra: Using Sinib through gaiaAssociation

Sinib is an R package developed by Boxiang Liu as an implementaion of their method for summing independent non-identical binomial random variables found here:

https://cran.r-project.org/web/packages/sinib/index.html

As part of the gaiaAssociation package the entire sinib package has been translated into Python and can be accessed through gaia. Its usage and the specific order of vairables in the function call is exactly the same as is found in its R documentation,which can be referenced if neccesary:

https://cran.r-project.org/web/packages/sinib/sinib.pdf

Below we provide an example of how to call the psinib function from gaiaAssociation. To see all functions accesible within gaiaAssociation you may type "gaiaAssociation.gaiaAssociation." into a code block and press Tab. This will provide a list of all functions within gaia.

In [3]:
gaiaAssociation.gaiaAssociation.psinib(1, [1,2], [.6,.3], lowerTail=False)

0.3634744120510757